##                                 有关幻影坦克技术的讲解
啥是幻影坦克? 幻影坦克就是, 一张黑白图片, 在黑色背景下和白色背景下能够显示出不同的图像.

它的原理就是控制像素的颜色和Alpha通道(不透明度), 来使显示的图像在不同背景下显示不同的颜色.

最基本的, 就是, 一张半透明的黑色薄膜, 如果在黑色的纸上, 你什么也看不出来, 但如果在白色纸上, 你可以看见, 它是灰色.

先同意讲Alpha值和像素的亮度统一至 0 - 1

###                                         基本原理
设: 这个图像的背景亮度为 bc, 这个像素的亮度为 pc, 不透明度为 pa, 则最终显示的颜色 oc 就是:
                           oc = pc * pa + bc * (1 - pa)

理解起来也简单, 还拿刚刚的例子来讲, 例如一个纯黑的半透明薄膜, 那他肯定:

1.只有一半的黑色能显示出来, 即： pc * pa

2.而背景色, 也有一半的颜色能够透过来. 即:   bc * (1 - pa)

3.总的颜色加起来, 也就是： pc * pa + bc * (1 - pa)

###                                         幻影坦克
当一个像素为白色背景时, 能够显示出一个特定的颜色 x, 当黑色背景时, 显示出 y, 故可得出一个公式：

  设颜色 x 的亮度为 xc, 颜色 y 的亮度为 yc, 这个像素的亮度为 zc, 不透明度为 za, 则满足:
 
$$
\begin{cases}
& \text{xc = za × zc + (1 − za)}\\
& \text{yc = za × zc}
\end{cases}
$$

稍微处理一下，则可以得到以下的公式：

$$
\begin{cases}
& \text{xc = yc + 1 × ( 1 − za )}\\
& \text{yc = xc − 1 × ( 1 − za)}\\
& \text{za = − [( xc − yc ) ÷ 1 ] + 1}
\end{cases}
$$
$$↓$$
$$
\begin{cases}
& \text{xc = yc + 1 − za}\\
& \text{yc = xc + za − 1}\\
& \text{za = yc − xc + 1}
\end{cases}
$$
 



由于 xc, yc, zc, za 都是小于等于1, 大于等于0的值, 所以:
$$
\begin{cases}
1 \geq xc \geq 0 \\
1 \geq yc \geq 0 \\
1 \geq za \geq 0
\end{cases}
$$
$$↓$$
$$
\begin{cases}
1 \geq yc + 1 - za \geq 0 \\
1 \geq xc + za - 1 \geq 0 \\
1 \geq yc - xc + 1 \geq 0
\end{cases}
$$
$$↓$$
$$
\begin{cases}
0 \geq yc - za \geq -1 \\
2 \geq yc + za \geq 1 \\
0 \geq yc - xc \geq -1
\end{cases}
$$
$$↓$$
$$
\begin{cases}
 za \geq yc \\
 xc + za \geq 1 \\
 xc \geq yc
\end{cases}
$$
最终得到的不等式, 则是我们的 x 和 y 需要满足的条件.

只有 x 和 y 像素满足这些条件, za 和 zc 才有值

所以我们最终的算式为：

$$
\begin{cases}
zc = \frac{yc}{za} \\
za = yc - xc + 1
\end{cases}
$$

由于ARGB的通道是0 - 255 ，所以我们需要将以进行转化：

$$
\begin{cases}
zc = \frac{yc}{\frac{za}{255}} = \frac{yc \times 255}{za} \\
za = yc - xc + 255
\end{cases}
$$

而经过刚才的推导，我们真正需要处理的只有：

$$ xc ≥ yc $$

## 代码实现

我们采用opencv库来对图像进行处理

首先我们可以定义一个函数对 xc ，yc 进行处理，使 xc ≥ yc ：

### 定义函数使 xc ≥ yc 

In [ ]:
import numpy as np

In [ ]:
def xc_max_than_yc(xc, yc, color_ratio=0.5):
    
    threshold = 255 * color_ratio
    xc = (xc / 255) * (255 - threshold) + threshold
    yc = (yc / 255) * threshold
    return xc, yc 

现在我们可以随机生成一个数组来测试一下

In [ ]:
xc = np.random.randint(255,size = (5,5))
yc = np.random.randint(255,size = (5,5))
xc_,yc_ = xc_max_than_yc(xc,yc)
xc,yc,xc_,yc_

上述函数对xc,yc进行处理，color_ratio参数用于调整xc，yc的颜色占比

其中xc，yc输入的都是图像的像素矩阵

### 效果图生成函数

In [ ]:
import cv2

def create_tank(white, black):
    # white表示白色背景时显示的图片
    # black表示黑色背景时显示的图片
    # 接下来对 透明度（za） 和 亮度（zc）进行处理
    za = black - white + 255
    zc = black.copy()
    idx = black != 0
    zc[idx] = black[idx] * 255 / za[idx]
    img = cv2.merge((zc, zc, zc, za))
    return img


这段代码定义了一个名为create_tank的函数，该函数接受两个参数white和black。函数的作用是创建一个坦克的图片。

在函数内部，首先计算透明度（za）和亮度（zc）的处理。

透明度的计算公式为：za = black - white + 255，这个公式通过将黑色背景的像素值减去白色背景的像素值，再加上255来计算透明度。

亮度的计算则是对黑色背景进行复制：zc = black.copy()。

接下来，通过使用布尔索引，找出黑色背景中不为0的像素，并将亮度进行调整。

具体而言，对于非零像素的位置，将其亮度乘以255再除以对应的透明度，以调整亮度值。

最后，使用OpenCV的cv2.merge()函数将调整后的亮度（zc）和透明度（za）合并成为一个4通道的图片。

返回的img即为创建的坦克图片。

### 接下来则是以具体的图片举例


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

img_false = cv2.imread('img1.jpg',0)
img_true = cv2.imread('img2.jpg',0)

x,y = img_true.shape
img_false = cv2.resize(img_false, (y, x))

xc, yc = xc_max_than_yc(img_false, img_true)
Img_output = create_tank(xc, yc)

cv2.imwrite("Img_output.png", Img_output)

In [ ]:
img = plt.imread('Img_output.png')
# 黑色背景
plt.figure(facecolor='black')
plt.imshow(img, cmap='gray')
plt.axis('off')
plt.title("Black Background",color='white')
plt.show()
# 白色背景
plt.figure(facecolor='white')
plt.imshow(img, cmap='gray')
plt.axis('off')
plt.title("White Background")
plt.show()

## 前端GUI开发
GUI开发部分相对简单，主要是先构思需要什么元件并打算好每个元件的位置。优秀的GUI还需要考虑色彩搭配等等，但我们的界面只是勉强能用，谈不上多优美，主要体验一下编写前端代码的过程。


### 利用工具进行页面布局
一个在线的[小工具](https://pytk.net/tkinter-helper/)，可以进行可视化的tkinter界面排版。在小工具内可以把几个元件安排到合适的位置。最终利用该工具排版得到一个比较满意的结果，通过工具提供的各个元件的尺寸的坐标生成GUI。

In [ ]:

import tkinter as tk
win = tk.Tk()
win.title("幻影坦克生成器")
win.geometry('600x500')

# Labels
title = tk.Label(master=win, text="幻影坦克生成器", anchor='center', font=('楷体',16))
title.place(x=220, y=0, width=165, height=40)
status = tk.Label(master=win,text="status",anchor='center', font=("楷体",16))
status.place(x=100,y=390,width=400,height=30)
Img1_L = tk.Label(master=win, text="图片1",anchor='center')
Img1_L.place(x=0, y=60, width=120, height=120)
Img2_L = tk.Label(master=win, text="图片2",anchor="center")
Img2_L.place(x=0,y=260,width=120,height=120)
Img_output_L =tk.Label(master=win, text="输出图片",anchor="center")
Img_output_L.place(x=470, y=160, width=120, height=120)
Address_1 = tk.Label(master=win,text="图片1地址",anchor='center', font=("楷体",12))
Address_1.place(x=190, y=90, width=80, height=30)
Address_2 = tk.Label(master=win, text="图片2地址",anchor='center', font=("楷体",12))
Address_2.place(x=190, y=290, width=80, height=30)

# Entrys
Txt1 = tk.Entry(master=win)
Txt1.place(x=140, y=60, width=180, height=30)
Txt2 = tk.Entry(master=win)
Txt2.place(x=140, y=260, width=180, height=30)

# Buttons
Cmd1 = tk.Button(master=win, text="导入上层掩体图", font=("楷体",10), command=lambda :selectfile(Txt1, Img1_L))
Cmd1.place(x=180, y=130, width=100, height=40)
Cmd2 = tk.Button(master=win, text="导入下层真实图", font=("楷体",10), command=lambda : selectfile(Txt2, Img2_L))
Cmd2.place(x=180, y=330, width=100, height=40)
Cmd3 = tk.Button(master=win, text="生成并保存图片", font=("楷体",10), command=lambda : generate())
Cmd3.place(x=340, y=190, width=120, height=40)

win.mainloop()

### 函数功能实现
我们希望在这个GUI中，点击“导入上层掩体图/下层真实图”之后，可以唤起一个文件浏览框，用户可以选择文件夹中的图片文件，随后返回文件地址到Entrys并在“图片1/2”中展示缩略图。点击“生成并保存图片”后，可以生成“幻影坦克”并在“输出图片”中展示缩略图，同时在当前目录保存图片。

功能很多，一一实现。

#### Selectfile(txt: tk.Entry, lab: tk.Label)
tk对于文件操作专门有一个类filedialog，其各种方法专门用来服务文件操作。我们需要寻找一个文件并获取文件地址，这用到了filedialog.askopenfilename()方法。该方法返回值为字符串形式的文件地址。随后将地址值赋给对应Entry。随后利用该地址用OpenCV读取对应图片，resize成合适的大小后保存在一个临时文件夹tmp中。由于这里需要临时文件夹tmp，可以在整个程序运行前先用OS库创建临时文件夹tmp。因为希望程序一次启动可以生成任意张“幻影坦克”，应该要将status的内容清空。

In [ ]:
import os
try:
    os.mkdir("tmp")
    print("Successfully Created direction 'tmp'!")
except FileExistsError:
    print("Successfully Created! diretion 'tmp'!")


In [ ]:
import tkinter as tk
from tkinter import filedialog
import cv2

def selectfile(txt: tk.Entry, lab: tk.Label):
    status.configure(text="")     #清空status，便于多次生成图片
    filepath = filedialog.askopenfilename()   #请求寻找一个图片文件,返回路径字符串
    txt.delete(0, 'end')
    txt.insert(0,filepath)   #将先前选择的路径填入对应地址栏
    
    tmp = cv2.imread(filepath, 0)    #将选中图片resize成(120,120)来展示缩略图
    tmp = cv2.resize(tmp, (120, 120))
    cv2.imwrite("tmp/tmp.png", tmp)
    panel = tk.Label(master=win)    #新建一个Label类panel引用photo，防止函数执行结束时释放内存导致photo丢失。
    panel.photo = tk.PhotoImage(file="tmp/tmp.png")
    lab.configure(image=panel.photo)
 
win = tk.Tk()
win.title("幻影坦克生成器")
win.geometry('600x500')

Txt1 = tk.Entry(master=win)
Txt1.place(x=140, y=60, width=180, height=30)

status = tk.Label(master=win,text="status",anchor='center', font=("楷体",16))
status.place(x=100,y=390,width=400,height=30)
Img1_L = tk.Label(master=win, text="图片1",anchor='center')
Img1_L.place(x=0, y=60, width=120, height=120)

Cmd1 = tk.Button(master=win, text="导入上层掩体图", font=("楷体",10), command=lambda :selectfile(Txt1, Img1_L))
Cmd1.place(x=180, y=130, width=100, height=40)

win.mainloop()

#### Generate()
我们希望在执行这个函数之后，OpenCV读取Entrys中指向的两张图片，并通过先前讲的一系列操作生成“幻影坦克”并同时生成“幻影坦克”的缩略图展示在“输出图片”中，同时更新status的显示文字。

In [ ]:
def generate():
    Img_1 = Txt1.get()
    Img_2 = Txt2.get()
    img_false = cv2.imread(Img_1, 0)     #掩体
    img_true = cv2.imread(Img_2, 0)     #底图
    x, y = img_true.shape
    img_false = cv2.resize(img_false, (y, x))
    xc, yc = xc_max_than_yc(img_false, img_true)
    Img_output = create_tank(xc, yc)
    
    tmp = cv2.resize(Img_output, (120,120))   #生成成品缩略图
    cv2.imwrite("tmp/Img_output.png", tmp)

    cv2.imwrite("Img_output.png", Img_output) 

    panel_ = tk.Label(master=win)
    panel_.photo = tk.PhotoImage(master=win, file="tmp/Img_output.png")
    Img_output_L.configure(image=panel_.photo)
    status.configure(text="成功生成！图片已保存至当前目录。")

### GUI主题更改美化
利用ttkbootstrap库调用预制的几个主题粗粗美化一下GUI

In [ ]:
import ttkbootstrap
#style = ttkbootstrap.Style(theme="darkly")    #利用ttkbootstrap快捷部署gui主题
#win = style.master

## 最后展示完整代码

In [ ]:
import tkinter as tk
from tkinter import filedialog
import ttkbootstrap
import cv2
import os

try:
    os.mkdir("tmp")
    print("Successfully Created Direction 'tmp'!")
except FileExistsError:
    print("Successfully Created Direction 'tmp'!")

style = ttkbootstrap.Style(theme="darkly")    #利用ttkbootstrap快捷部署gui主题
win = style.master

def xc_max_than_yc(xc, yc, color_ratio=0.5):
    threshold = 255 * color_ratio
    xc = (xc / 255) * (255 - threshold) + threshold
    yc = (yc / 255) * threshold
    return xc, yc 

def create_tank(white, black):
    za = black - white + 255
    zc = black.copy()
    idx = black != 0
    zc[idx] = black[idx] * 255 / za[idx]
    img = cv2.merge((zc, zc, zc, za))
    return img

def selectfile(txt: tk.Entry, lab: tk.Label):
    status.configure(text="")     #清空status，便于多次生成图片
    filepath = filedialog.askopenfilename()   #请求寻找一个图片文件,返回路径字符串
    txt.delete(0, 'end')
    txt.insert(0,filepath)   #将先前选择的路径填入对应地址栏
    
    tmp = cv2.imread(filepath, 0)    #将选中图片resize成(120,120)来展示缩略图
    tmp = cv2.resize(tmp, (120, 120))
    cv2.imwrite("tmp/tmp.png", tmp)
    panel = tk.Label(master=win)    #新建一个Label类panel引用photo，防止函数执行结束时释放内存导致photo丢失。
    panel.photo = tk.PhotoImage(file="tmp/tmp.png")
    lab.configure(image=panel.photo)

def generate():
    Img_1 = Txt1.get()
    Img_2 = Txt2.get()
    img_false = cv2.imread(Img_1, 0)     #掩体
    img_true = cv2.imread(Img_2, 0)     #底图
    x, y = img_true.shape
    img_false = cv2.resize(img_false, (y, x))
    xc, yc = xc_max_than_yc(img_false, img_true)
    Img_output = create_tank(xc, yc)
    
    tmp = cv2.resize(Img_output, (120,120))   #生成成品缩略图
    cv2.imwrite("tmp/Img_output.png", tmp)

    cv2.imwrite("Img_output.png", Img_output) 

    panel_ = tk.Label(master=win)
    panel_.photo = tk.PhotoImage(master=win, file="tmp/Img_output.png")
    Img_output_L.configure(image=panel_.photo)
    status.configure(text="成功生成！图片已保存至当前目录。")

# win = tk.Tk()
win.title("幻影坦克生成器")
win.geometry('600x500')

# Labels
title = tk.Label(master=win, text="幻影坦克生成器", anchor='center', font=('楷体',16))
title.place(x=220, y=0, width=165, height=40)
status = tk.Label(master=win,text="",anchor='center', font=("楷体",16))
status.place(x=100,y=390,width=400,height=30)
Img1_L = tk.Label(master=win, text="图片1",anchor='center')
Img1_L.place(x=0, y=60, width=120, height=120)
Img2_L = tk.Label(master=win, text="图片2",anchor="center")
Img2_L.place(x=0,y=260,width=120,height=120)
Img_output_L =tk.Label(master=win, text="输出图片",anchor="center")
Img_output_L.place(x=470, y=160, width=120, height=120)
Address_1 = tk.Label(master=win,text="图片1地址",anchor='center', font=("楷体",12))
Address_1.place(x=190, y=90, width=80, height=30)
Address_2 = tk.Label(master=win, text="图片2地址",anchor='center', font=("楷体",12))
Address_2.place(x=190, y=290, width=80, height=30)

# Entrys
Txt1 = tk.Entry(master=win)
Txt1.place(x=140, y=60, width=180, height=30)
Txt2 = tk.Entry(master=win)
Txt2.place(x=140, y=260, width=180, height=30)

# Buttons
Cmd1 = tk.Button(master=win, text="导入上层掩体图", font=("楷体",10), command=lambda :selectfile(Txt1, Img1_L))
Cmd1.place(x=180, y=130, width=100, height=40)
Cmd2 = tk.Button(master=win, text="导入下层真实图", font=("楷体",10), command=lambda : selectfile(Txt2, Img2_L))
Cmd2.place(x=180, y=330, width=100, height=40)
Cmd3 = tk.Button(master=win, text="生成并保存图片", font=("楷体",10), command=lambda : generate())
Cmd3.place(x=340, y=190, width=120, height=40)

win.mainloop()